Implement a ML classifier based on decision trees (in Python using Jupyter notebook).

### Task 1

##### A
Implement the basic decision tree learning procedure. First, you will implement a train procedure for your decision tree algorithm. This train procedure will use the information gain criterion to decide on splits, recursively until you have leaves of only one class. 

In [333]:
import pandas as pd
import numpy as np
import math

training_data = pd.read_csv("./adult_train_data.csv")
#print(training_data.shape)


def entropy(p_positive, p_negative):
    if p_positive == 0 or p_negative == 0: #log of 0 is undefined, assign beforehand to avoid error #when class shows only one type (100%)
        return 0 #For ppos or pneg = 1, the entropy will be zero due to log2(1) = 0
    return -(p_positive * math.log2(p_positive) + p_negative * math.log2(p_negative))


def informational_gain(entropy_before_split, p_c__g, entropy_category):
    info_gain = entropy_before_split
    for i in range(len(entropy_category)):
        info_gain -= p_c__g.iloc[i] * entropy_category.iloc[i] #.iloc[] allows to access DataFrame or Series like a list.
    return info_gain


#function to calculate entropy of a dataset
def calculate_entropy(training_data, target):
    n = len(training_data)
    p_positive = training_data[target].mean()
    p_negative = 1 - p_positive
    return entropy(p_positive, p_negative)


def build_decision_tree(training_data, features, target, stopping_depth = None, current_depth = 0):
    # Base Case 1: If all labels are the same, return the class
    if len(training_data[target].unique()) == 1: #if there's only one unique value, this means all examples in this subset belong to the same class (either 0 or 1). (0% impurity) #in this case, we return that class as a leaf node, because there's no need to split further.
        return training_data[target].iloc[0]  #return the label
    
    # Base Case 2: If no features left to split, return the most common class
    if len(features) == 0:
        return training_data[target].mode()[0]  #return most frequent class label in the current subset.
    

    #Base Case 3: If we reached the stopping depth, return the most common class
    if stopping_depth is not None and current_depth >= stopping_depth:
        return training_data[target].mode()[0]  # return most frequent class label

    
    iglist = {}  #dictionary to store info gains for each feature
    entropy_before_split = calculate_entropy(training_data, target)
    
    for feature in features: #iterating through each column to calculate IG for each feature
        #group by column values and compute counts of 1s and 0s in target column
        g = training_data.groupby(feature)[target] #groups y based on unique values of colvalues
        gc = g.value_counts() #for each group, counts how many y is 1
        grouped = gc.unstack(fill_value=0) #.groupby() gives MultiIndex, .unstack() converts to DataFrame #fill_value ensures that if a certain combination is missing, then fills that cell with 0
        
        #probability of each class per group
        p_c__g = grouped.sum(axis=1) / len(training_data)  #axis=0: summing along the columns, axis=1: summing along the rows
        
        entropy_category = pd.Series(index=grouped.index)  # Create an empty Series with the same index as grouped
        for group in grouped.index:
            p_pos = grouped.loc[group, 1] / grouped.loc[group].sum()  #P(positive class)
            p_neg = grouped.loc[group, 0] / grouped.loc[group].sum()  #P(negative class)
            entropy_category[group] = entropy(p_pos, p_neg)  #Entropy for this group
        
        info_gain = informational_gain(entropy_before_split, p_c__g, entropy_category)
        iglist[feature] = info_gain #store entropy for each category
    
    
    #choose the feature with the highest information gain
    best_feature = max(iglist, key=iglist.get)
    
    #tree structure
    tree = {best_feature: {}}
    
    #recursively split the data based on the best feature
    for value in training_data[best_feature].unique():
        subset = training_data[training_data[best_feature] == value]  #filter rows for this feature value
        new_features = [f for f in features if f != best_feature]  #remove best feature from future splits
        tree[best_feature][value] = build_decision_tree(subset, new_features, target, stopping_depth, current_depth + 1)  #recursive call, increment depth by 1

    return tree

In [334]:
'''
def print_tree(tree, indent=""):
    for key, value in tree.items():
        if isinstance(value, dict):
            print(f"{indent}{key}:")
            print_tree(value, indent + "  ")
        else:
            print(f"{indent}{key}: {value}")
'''
            
def print_tree(tree, indent="", level=0, max_lines=100, current_line=0):
    if current_line >= max_lines:
        return current_line  #stop printing if we've reached the max lines
    
    for key, value in tree.items():
        if isinstance(value, dict):
            print(f"{indent}{key}:")
            current_line = print_tree(value, indent + "  ", level + 1, max_lines, current_line)
        else:
            print(f"{indent}{key}: {value}")
            current_line += 1
        
        if current_line >= max_lines:
            return current_line  #stop if we've printed enough lines

    return current_line

max_lines = 100

features = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
target = "income_more50K"
tree = build_decision_tree(training_data, features, target)
print_tree(tree, max_lines = max_lines) #first 100 levels of full complete decision tree with no depth limitation
#print_tree(tree)


relationship:
  Not-in-family:
    education:
      Bachelors:
        age:
          30-39:
            hours_per_week:
              35-40:
                sex:
                  Male:
                    occupation:
                      Adm-clerical:
                        fnlwgt:
                          <100K: 0
                          100K-200K: 0
                          200K-300K:
                            workclass:
                              Private: 0
                              Federal-gov: 1
                          >300K: 1
                      Tech-support: 0
                      Exec-managerial:
                        marital_status:
                          Never-married:
                            fnlwgt:
                              >300K:
                                workclass:
                                  Self-emp-not-inc: 0
                                  Private:
                                    education_num:
                    

100

The trained decision tree, built without depth limitations, using the training data. The tree fully splits the data until each leaf contains only one class. As a result, the tree is highly complex and suggests overfitting due to excessive branching. The tree is large and code takes over 1 minute to run.

##### B
Implement tree depth control as a means of controlling the model complexity. Trained tree using training data at stopping level 2, 3, 4.  

In [335]:
#test for different stopping depths
print("Decision Tree at Depth 2:")
tree2 = build_decision_tree(training_data, features, target, 2)
print_tree(tree2, max_lines = max_lines)

Decision Tree at Depth 2:
relationship:
  Not-in-family:
    education:
      Bachelors: 0
      HS-grad: 0
      9th: 0
      Masters: 0
      Assoc-acdm: 0
      Assoc-voc: 0
      Some-college: 0
      Doctorate: 1
      10th: 0
      Preschool: 0
      11th: 0
      Prof-school: 1
      7th-8th: 0
      5th-6th: 0
      12th: 0
      1st-4th: 0
  Husband:
    education:
      Bachelors: 1
      11th: 0
      HS-grad: 0
      Some-college: 0
      7th-8th: 0
      Doctorate: 1
      9th: 0
      Assoc-acdm: 0
      Assoc-voc: 0
      5th-6th: 0
      Masters: 1
      Prof-school: 1
      10th: 0
      1st-4th: 0
      12th: 0
      Preschool: 0
  Wife:
    occupation:
      Prof-specialty: 1
      Exec-managerial: 1
      Adm-clerical: 0
      Other-service: 0
      Sales: 0
      Craft-repair: 0
      Machine-op-inspct: 0
      Transport-moving: 0
      Tech-support: 1
      Farming-fishing: 0
      Handlers-cleaners: 0
      Priv-house-serv: 0
      Protective-serv: 0
  Own-child:

81

At depth 2, the decision tree primarily splits based on the "relationship" feature, followed by "education," "occupation," or "age" depending on the branch. This results in a more generalized model with fewer branches, reducing complexity compared to the full tree. The limited depth may lead to underfitting as the tree lacks deeper feature interactions.

In [336]:
print("Decision Tree at Depth 3:")
tree3 = build_decision_tree(training_data, features, target, 3)
print_tree(tree3, max_lines = max_lines)

Decision Tree at Depth 3:
relationship:
  Not-in-family:
    education:
      Bachelors:
        age:
          30-39: 0
          50-59: 0
          20-29: 0
          40-49: 0
          >=60: 0
      HS-grad:
        occupation:
          Handlers-cleaners: 0
          Exec-managerial: 0
          Sales: 0
          Craft-repair: 0
          Adm-clerical: 0
          Other-service: 0
          Prof-specialty: 0
          Transport-moving: 0
          Farming-fishing: 0
          Machine-op-inspct: 0
          Protective-serv: 0
          Tech-support: 0
          Priv-house-serv: 0
          Armed-Forces: 0
      9th:
        occupation:
          Other-service: 0
          Sales: 0
          Machine-op-inspct: 0
          Priv-house-serv: 0
          Handlers-cleaners: 0
          Craft-repair: 0
          Adm-clerical: 0
          Transport-moving: 0
          Farming-fishing: 0
          Prof-specialty: 0
          Exec-managerial: 0
          Protective-serv: 0
      Masters:
   

100

In [337]:
print("Decision Tree at Depth 4:")
tree4 = build_decision_tree(training_data, features, target, 4)
print_tree(tree4, max_lines = max_lines)

Decision Tree at Depth 4:
relationship:
  Not-in-family:
    education:
      Bachelors:
        age:
          30-39:
            hours_per_week:
              35-40: 0
              >40: 0
              <35: 0
          50-59:
            occupation:
              Exec-managerial: 1
              Prof-specialty: 0
              Sales: 0
              Adm-clerical: 0
              Other-service: 0
              Transport-moving: 0
              Craft-repair: 0
              Protective-serv: 1
              Tech-support: 0
              Farming-fishing: 0
              Machine-op-inspct: 0
          20-29:
            hours_per_week:
              35-40: 0
              >40: 0
              <35: 0
          40-49:
            occupation:
              Exec-managerial: 0
              Sales: 0
              Prof-specialty: 0
              Craft-repair: 0
              Other-service: 0
              Protective-serv: 0
              Adm-clerical: 0
              Handlers-cleaners: 0
     

100

As the tree depth increases, it captures more complex patterns in the training data. At depth 3, the tree expands further into subcategories within each relationship type, refining its predictions while still maintaining some generalization. At depth 4, the tree grows significantly, allowing for more complex decision branches, but also increasing the risk of overfitting by tailoring itself too closely to the training data.

##### C
Implement a test procedure for your decision tree algorithm. Takes the new data (test data) and the trained decision tree model (the full complete decision tree you trained in A or one of the decision trees with depth control you trained in B) and returns a prediction for each example in the new data.


In [338]:
test_data = pd.read_csv("./adult_test_data.csv")
#print(test_data.shape)


def predict(tree, sample): #sample: A single row from the test data, tree needs to be dictionary
    """Recursively traverse the tree to predict a class label for a given sample. And in the end this function arrives at the leaf, and returns a 1 or a 0"""
    if not isinstance(tree, dict):
        return tree  #if tree is not a dictionary, it must be a leaf node (either 0 or 1).
    
    feature = next(iter(tree))  #next(iter(tree)) retrieves the first key (the feature name)
    value = sample[feature] #content of column in test data
    
    if value in tree[feature]: #checks if the decision tree has a branch corresponding to value
        return predict(tree[feature][value], sample)  #Moves down the tree to the next node.
    else:
        return None  #if value doesn't exist in the tree
#returns a prediction whether 1 or 0


def f1_score(tree, td, target):
    predictions = [] #list of predictions (made by tree from training_data)
    
    for _, row in td.iterrows(): #iterates over each row of the test dataset. #td.iterrows() returns (index, row), but _ ignores the index.
        prediction = predict(tree, row)  #call predict function for each row
        predictions.append(prediction)
    
    actual_values = td[target].to_numpy()  #actual values from the target column #.to_numpy() converts the entire DataFrame into a NumPy array. Each column becomes an element in the array.
    #list of actual values (from the test_data)

    TP = 0 #True Positives
    FP = 0 #False Positives
    FN = 0 #False Negatives
    TN = 0 #True Negatives

    for i in range(len(predictions)):
        if predictions[i] == 1 and actual_values[i] == 1:
            TP += 1  #True Positive: predicted 1, actual 1
        elif predictions[i] == 1 and actual_values[i] == 0:
            FP += 1  #False Positive: predicted 1, actual 0
        elif predictions[i] == 0 and actual_values[i] == 1:
            FN += 1  #False Negative: predicted 0, actual 1
        elif predictions[i] == 0 and actual_values[i] == 0:
            TN += 1  #True Negative: predicted 0, actual 0
    
    # Calculate Precision and Recall
    if TP + FP > 0: #avoid division by zero
        precision = TP / (TP + FP)  #Precision: TP / (TP + FP)
    else:
        precision = 0 
    
    if TP + FN > 0: #avoid division by zero
        recall = TP / (TP + FN)  #Recall: TP / (TP + FN)
    else:
        recall = 0

    
    #Accuracy: TP + TN / TOTAL
    accuracy = (TP + TN) / len(td)

    # Calculate F1-Score: 2TP / (2TP + FP + FN)
    if (2*TP + FP + FN > 0): #avoid division by zero
        f1 = (2 * TP) / (2*TP + FP + FN)  #Recall: TP / (TP + FN)
    else:
        f1 = 0 
    
    return f1, accuracy


f1, accuracy = f1_score(tree2, test_data, target)
print("Accuracy on test data (depth 2):", round(accuracy*100, 4), "%")
print("F1-Score (depth 2): ", round(f1, 4))

f1, accuracy = f1_score(tree3, test_data, target)
print("Accuracy on test data (depth 3):", round(accuracy*100, 4), "%")
print("F1-Score (depth 3): ", round(f1, 4))

f1, accuracy = f1_score(tree4, test_data, target)
print("Accuracy on test data (depth 4):", round(accuracy*100, 4), "%")
print("F1-Score (depth 4): ", round(f1, 4))

Accuracy on test data (depth 2): 81.5206 %
F1-Score (depth 2):  0.5317
Accuracy on test data (depth 3): 82.0518 %
F1-Score (depth 3):  0.59
Accuracy on test data (depth 4): 81.0292 %
F1-Score (depth 4):  0.5981


Increasing depth from 2 to 3 slightly improves both accuracy and F1-score, suggesting the model benefits from deeper splits.
However, at depth 4, accuracy drops slightly, while the F1-score remains similar to depth 3. This could indicate overfitting and adding more depth does not necessarily improve test performance.

Tree with depth 3 might be a good balance between model complexity and generalization.

However Evaluating solely on test performance isn’t enough to confirm overfitting. To properly assess it, we need to look at the relationship between training and validation performance, ideally using cross-validation. A large gap between training and validation accuracy is a clearer sign of overfitting than test accuracy alone.

### Task 2
##### 1. Effect of Changing the Splitting Criterion

In my implementation, the decision tree uses information gain as the splitting criterion. Information Gain prefers splits that maximize class separation by reducing entropy.

Another splitting criterion is Gini impurity, which measures how "mixed up" the data is at a certain point in the decision tree. If a node has a mix of different classes, the impurity is high. If a node has mostly one class, the impurity is low. Using Gini impurity, when the decision tree splits the data, it tries to reduce impurity—meaning it wants to make the groups as "pure" as possible. Gini tends to prefer splits that make big groups, rather than breaking data into many small groups.

Using Gini impurity may lead to different tree structures since it prioritizes larger homogeneous groups.
Also accuracy may change but would be similar to current accuracy (made with informational gain splitting criterion).


In [339]:
f1, accuracy = f1_score(tree2, training_data, target)
print("Accuracy on training data (depth 2):", round(accuracy*100, 4), "%")
print("F1-Score (depth 2): ", round(f1, 4))

f1, accuracy = f1_score(tree3, training_data, target)
print("Accuracy on training data (depth 3):", round(accuracy*100, 4), "%")
print("F1-Score (depth 3): ", round(f1, 4))

f1, accuracy = f1_score(tree4, training_data, target)
print("Accuracy on training data (depth 4):", round(accuracy*100, 4), "%")
print("F1-Score (depth 4): ", round(f1, 4))

Accuracy on training data (depth 2): 81.7281 %
F1-Score (depth 2):  0.5427
Accuracy on training data (depth 3): 82.8023 %
F1-Score (depth 3):  0.605
Accuracy on training data (depth 4): 84.6292 %
F1-Score (depth 4):  0.661


##### 2. Can the Test Procedure Indicate Overfitting or Underfitting?

Yes, the test procedure helps identify overfitting or underfitting by comparing model performance on training data vs. test data.
If the model performs well against the training data (high accuracy or F1-score) but has much lower accuracy or F1-score against the test data, this could indicate overfitting.
If both training and test accuracy are low, the model is too simple to capture patterns in the data. A shallow tree may underfit if it does not split enough to separate different classes effectively.

Depth 2: Training and test accuracy are quite similar (~81.7% vs. ~81.5%), meaning the model is not overfitting.
Depth 3: Training accuracy increases (~82.8%) and test accuracy slightly improves (~82.0%), so the tree is generalizing well.
Depth 4: Training accuracy jumps to 84.6%, but test accuracy drops (~81.0%). This suggests overfitting—the model is too tailored to training data and loses generalization.

These results suggest that tree with depth 3 provides the best balance between complexity and performance, while deeper trees may begin to overfit to the training data.

"""Notes:
Train the decision tree model using your training data.
Once the model is trained, you test it using the test data (which the model has never seen before).
The test data is used for evaluation, not for training, to see how well your tree performs outside of the training set.
Compare the predictions made by your trained decision tree on the test data to the actual values in the test set.
Then use training data for evaluation, compare accuracy with testing data evaluation to see if the model underfit/overfit.
"""
